# Get the Sector and Industry for all the stocks

In [10]:
import numpy as np
import pandas as pd

import requests
import string
from time import sleep

# To print nicely
from rich.jupyter import print
from rich.traceback import install
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fd314093f70>>

In [11]:
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv()

API_KEY = os.getenv("ALPHA_VANTAGE_API")

Get all the stock tickers for which we need information

In [12]:
all_stocks = pd.read_csv("../all_stock_info_needed.csv")
all_stocks.head()

,Ticker,Company Name,ISIN Code
0,A,Agilent Technologies Inc.,US00846U1016
1,AA,Alcoa Inc,US0138171014
2,AAL,American Airlines Group Inc,US02376R1023
3,AAP,Advance Auto Parts Inc,US00751Y1064
4,AAPL,Apple Inc.,US0378331005


In [13]:
def get_company_overview(ticker):
    while True:
        url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        
        if len(data.keys()) == 0:
            raise Exception(f"returned {data}")
        elif len(data.keys()) == 1:
            print(f"[green]Retrying {ticker} after 1 min, got {data}[/green]")
            sleep(60)
            continue
        break
    
    return data

def strip_string(text):
    return text.translate(str.maketrans('', '', string.punctuation)).strip().lower()

def string_similar(str1, str2):
    return strip_string(str1) == strip_string(str2)

In [14]:
stock_info = pd.DataFrame(columns=["Ticker", "Name", "Sector", "Industry"])

In [15]:
manual_check = []
skip = []

errors = ""

for _, row in all_stocks.iterrows():
    if stock_info[stock_info["Ticker"] == row["Ticker"]].shape[0] > 0 or row["Ticker"] in skip:
        continue
    ticker = row["Ticker"]
    
    try:
        company_info = get_company_overview(ticker)
    except Exception as e:
        errors += f"{ticker} | {row['Company Name']} | {e}\n"
        skip.append(ticker)
        continue
    
    
    new_row = pd.DataFrame([{
        "Ticker": ticker,
        "Name": company_info.get("Name"),
        "Sector": company_info.get("Sector"),
        "Industry": company_info.get("Industry")
    }])
    stock_info = pd.concat([stock_info, new_row], ignore_index=True)
    
    if ticker not in manual_check and company_info.get("Name") != row["Company Name"]:
        print(f"{ticker} | {row['Company Name']} | {company_info.get('Name')}")

print(errors)

A | Agilent Technologies Inc. | Agilent Technologies Inc

AA | Alcoa Inc | Alcoa Corp

AAL | American Airlines Group Inc | American Airlines Group

AAPL | Apple Inc. | Apple Inc

ABBV | AbbVie Inc. | AbbVie Inc

...

GDDY | GoDaddy Inc | Godaddy Inc

PLTR | Palantir Technologies Inc | Palantir Technologies Inc

ERIE | Erie Indemnity Co | Erie Indemnity Company

AMTM | Amentum Holdings Inc | Amentum Holdings Inc.

ABI | Applied Biosystems Inc. | returned {}
ABS | Albertson's Inc. | returned {}
ABX | Barrick Gold Corp. | returned {}
ACAS | American Capital Ltd | returned {}
ACKH | Armstrong Holding Inc | returned {}
ACS | Affiliated Computer Services | returned {}
ACV | Alberto-Culver Co. | returned {}
SXCL | Steel Excel Inc | returned {}
ADS | Alliance Data Systems Corp. | returned {}
AET | Aetna Inc. | returned {}
AFS.A | Associates First Capital Corp | returned {}
AGC | American General Insurance Co | returned {}
AGN | Allergan plc | returned {}
AGN (Old) | Allergan Inc. (Old) | returned {}
AKS | AK Steel Holding Corp. | returned {}
AL (Alc) | Alcan Inc. | returned {}
ALTR (Alt) | Altera Corp. | returned {}
ALXN | Alexion Pharmaceuticals Inc | returned {}
AMCC | Applied Micro Circuits Corp. | returned {}
AMR (AMR) | AMR Corp. | returned {}
ANDW | Andrew Corp. | returned {}
ANR | Alpha Natural Resources Inc | returned {}
APC | Anadarko Petroleum Corp. | returned {}
APCC | American Power Conversion Corp | returned {}
APOL | Apollo Education Group | returned {}
ARC (Atl) | Atlantic Richfield Co | returned {}
ARG | Airgas Inc. | returned {}
ASN | Archstone-Smith Trust | returned {}
AT (All) | Alltel Corp | returned {}
ATVI | Activision Blizzard Inc | returned {}
AV | Avaya Inc. | returned {}
AVP | Avon Products Inc | returned {}
AW | Allied Waste Industries Inc. | returned {}
AWE | AT&T Wireless | returned {}
AYE | Allegheny Energy Inc. | returned {}
AZA.A | ALZA Corp. | returned {}
BBBY | Bed Bath & Beyond Inc | returned {}
BCR | C. R. Bard Inc | returned {}
BDK | Black & Decker Manufacturing | returned {}
BEAM (Sun) | Beam Suntory Inc | returned {}
BF.B | Brown-Forman Corp. | returned {}
BFO | BestFoods | returned {}
BGEN | Biogen NV | returned {}
BGG | Briggs & Stratton Corp | returned {}
BIG | Big Lots Inc. | returned {}
BJS | BJ Services Co. | returned {}
BLS | BellSouth Corp. | returned {}
BMC | BMC Software Inc | returned {}
BMET | Biomet Inc. | returned {}
BMS | Bemis Company Inc | returned {}
BNI | Burlington Northern Santa Fe Corp | returned {}
BOL | Bausch & Lomb Inc. | returned {}
BR (OLD) | Burlington Resources | returned {}
BRCM | Broadcom Corp. | returned {}
BRK.B | Berkshire Hathaway Inc | returned {}
BRL | Barr Pharmaceuticals Inc. | returned {}
BS | Bethlehem Steel Corp | returned {}
BSC | Bear Stearns Co | returned {}
BUD (Anh) | Anheuser-Busch Inc. | returned {}
BVSN | BroadVision Inc. | returned {}
BXLT | Baxalta | returned {}
CA | CA Inc | returned {}
CAM | Cameron International Corp | returned {}
CB (Old) | Chubb Corp. (Old) | returned {}
CBE | Cooper Industries plc | returned {}
CBH | Commerce Bancorp (New Jersey) | returned {}
CBS (Old) | CBS Corp. (Old) | returned {}
CBSS | Compass Bancshares Inc | returned {}
CC (Circ) | Circuit City Stores Inc. | returned {}
CCE | Coca-Cola European Partners plc | returned {}
CE (Con) | Concord EFS Inc. | returned {}
CEG (Old) | Constellation Energy Group Inc (Old) | returned {}
CELG | Celgene Corp. | returned {}
CEN | Ceridian Corp. | returned {}
CEPH | Cephalon Inc. | returned {}
CERN | Cerner Corp | returned {}
CF (Char) | Charter One Financial Inc | returned {}
CFC | Countrywide Financial Corp. | returned {}
CFN | CareFusion Corp | returned {}
CGP | Coastal Corp. | returned {}
CHA | Champion International Corp | returned {}
CHIR | Chiron Corp. | returned {}
CIN | Cinergy Corp. | returned {}
CIT | CIT Group Inc. | returned {}
CMCSK | Comcast Class A Spec. | returned {}
CMCSK (Old) | Comcast Corp (Old) | returned {}
CMVT | Comverse Technology Inc | returned {}
CMX | Caremark Rx | returned {}
CNG | Consolidated Natural Gas Co | returned {}
CNXT | Conexant Systems | returned {}
COC.B | Conoco Inc. | returned {}
COL | Rockwell Collins Inc | returned {}
COMS | 3Com Corp. | returned {}
COV | Covidien Plc | returned {}
CPGX | Columbia Pipeline Group Inc. | returned {}
CPN | Calpine Corp. | returned {}
CPQ | Compaq Computer Corp | returned {}
CPWR | Compuware Corp | returned {}
CSC 

In [ ]:
all_stocks[all_stocks["Ticker"] == "ABX"]

,Ticker,Company Name,ISIN Code
11,ABX,Barrick Gold Corp.,CA0679011084


In [23]:
get_company_overview("ACI")

{'Symbol': 'ACI',
 'AssetType': 'Common Stock',
 'Name': 'Albertsons Companies',
 'Description': 'Albertsons Companies, Inc. participates in the pharmacy and food operation in the United States. ',
 'CIK': '1646972',
 'Exchange': 'NYSE',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TRADE & SERVICES',
 'Industry': 'RETAIL-GROCERY STORES',
 'Address': '250 PARKCENTER BLVD., BOISE, ID, US',
 'OfficialSite': 'https://www.albertsonscompanies.com',
 'FiscalYearEnd': 'February',
 'LatestQuarter': '2024-08-31',
 'MarketCapitalization': '10746905000',
 'EBITDA': '3863700000',
 'PERatio': '10.85',
 'PEGRatio': '2.836',
 'BookValue': '5.21',
 'DividendPerShare': '0.48',
 'DividendYield': '0.0259',
 'EPS': '1.71',
 'RevenuePerShareTTM': '137.96',
 'ProfitMargin': '0.0125',
 'OperatingMarginTTM': '0.0227',
 'ReturnOnAssetsTTM': '0.0505',
 'ReturnOnEquityTTM': '0.381',
 'RevenueTTM': '79713698000',
 'GrossProfitTTM': '21755600000',
 'DilutedEPSTTM': '1.71',
 'QuarterlyEarningsGrowthYOY': '-0.4

In [22]:
ticker = "ACI"
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&apikey={API_KEY}&outputsize=full"
response = requests.get(url)
data = response.json()
data['Time Series (Daily)'].keys()

dict_keys(['2024-11-05', '2024-11-04', '2024-11-01', '2024-10-31', '2024-10-30', '2024-10-29', '2024-10-28', '2024-10-25', '2024-10-24', '2024-10-23', '2024-10-22', '2024-10-21', '2024-10-18', '2024-10-17', '2024-10-16', '2024-10-15', '2024-10-14', '2024-10-11', '2024-10-10', '2024-10-09', '2024-10-08', '2024-10-07', '2024-10-04', '2024-10-03', '2024-10-02', '2024-10-01', '2024-09-30', '2024-09-27', '2024-09-26', '2024-09-25', '2024-09-24', '2024-09-23', '2024-09-20', '2024-09-19', '2024-09-18', '2024-09-17', '2024-09-16', '2024-09-13', '2024-09-12', '2024-09-11', '2024-09-10', '2024-09-09', '2024-09-06', '2024-09-05', '2024-09-04', '2024-09-03', '2024-08-30', '2024-08-29', '2024-08-28', '2024-08-27', '2024-08-26', '2024-08-23', '2024-08-22', '2024-08-21', '2024-08-20', '2024-08-19', '2024-08-16', '2024-08-15', '2024-08-14', '2024-08-13', '2024-08-12', '2024-08-09', '2024-08-08', '2024-08-07', '2024-08-06', '2024-08-05', '2024-08-02', '2024-08-01', '2024-07-31', '2024-07-30', '2024-07-

In [ ]:
import math
list_of_floats = [1.1, 2.2, 3.3]
stuff = map(round, list_of_floats)

del print
print(stuff) # Error

pw = math.pow
pw(16, 7)

ls = [round, input, sorted]

def pick(b):
    if b:
        return sorted
    else:
        return reversed

def quadratic(a, b, c):
    df = (b*b - 4*a*c)**0.5
    r1 = (-b + df) / (2 * a)
    r2 = (-b - df) / (2 * a)
    return r1, r2
first, second = quadratic(1, 4, 2)
# first = -0.58
# second = -3.41

def rectangle_properties(length, width):
    area = length * width
    perimeter = 2 * (length + width)
    diagonal = (length**2 + width**2)**0.5
    return area, perimeter, diagonal

# Sample function call
area, perimeter, diagonal = rectangle_properties(5, 12)

# Expected Output
# area = 60
# perimeter = 34
# diagonal ≈ 13.0


num = 2
def print_num():
    print(num)  # Prints 2
    
def make_num():
    num = 2
make_num()    
print(num)  # Causes NameError

def test_function():
    print(sum)  
# Here, sum refers to the built-in function <built-in function sum>

count = 0
def increment():
    global count
    count += 1

In [5]:
def test_function():
    print("Local namespace:", locals())
x = 100
var = 'apple'
print("Global namespace:", globals())
print("Namespace of a list object:", vars([1, 2, 3]))

Global namespace: {'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', "def test_function():\n    print(locals())\nx = 100\nvar = 'apple'\nprint(globals())", "def test_function():\n    print(locals()) # {'y': 50}\nx = 100\nvar = 'apple'\ntest_function()\nprint(globals())\n# {'__name__': '__main__', '__doc__': None, '__package__': None, '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7fbf94919030>, '__spec__': None, '__annotations__': {}, '__builtins__': <module 'builtins' (built-in)>, '__file__': '/mnt/d/Code/Acadmic/CS 682 -Neural Network/Project/data/processing/Stock Market Data/test.py', '__cached__': None, 'test_function': <function test_function at 0x7fbf9497e4d0>, 'x': 100, 'var': 'apple'}\nprint(vars(x))", "def test_function():\n    print(loca

TypeError: vars() argument must have __dict__ attribute

In [ ]:
# Filters out all even numbers
def div_2(x):
    return x % 2 != 0

my_list = [1, 2, 3, 4, 5]
filter_result = list(filter(div_2, my_list))
print(filter_result)  # [1, 3, 5]


In [5]:
my_dict = {
    'name': 'John',
    'details': {
        'favorite_numbers': [1, 2, 3],
        'favorite_colors': ['red', 'blue', 'green']
    },
    'hobbies': ['reading', 'coding', 'gaming']
}

print(my_dict)

import pprint

pprint.pprint(my_dict)

from pprint import pprint as pp
pp(my_dict)

import datetime
today = datetime.date.today()


from datetime import date
today = date.today()


from math import sqrt as s, pi as p
result = s(16)
print(f"The value of Pi is {p}")

{'name': 'John', 'details': {'favorite_numbers': [1, 2, 3], 'favorite_colors': ['red', 'blue', 'green']}, 'hobbies': ['reading', 'coding', 'gaming']}
{'details': {'favorite_colors': ['red', 'blue', 'green'],
             'favorite_numbers': [1, 2, 3]},
 'hobbies': ['reading', 'coding', 'gaming'],
 'name': 'John'}
{'details': {'favorite_colors': ['red', 'blue', 'green'],
             'favorite_numbers': [1, 2, 3]},
 'hobbies': ['reading', 'coding', 'gaming'],
 'name': 'John'}
The value of Pi is 3.141592653589793


In [6]:
from datetime import date as d
today = d.today()


In [8]:
import datetime
formats = ["nice", "slashes"]

def get_date_nice(date):
    return f"{date.strftime('%B')} {date.day}, {date.year}"

def get_with_slashes(date):
    return f"{date.month}/{date.day}/{date.year}"

def print_date(date, format):
    if format == "nice":
        print(get_date_nice(date))
    elif format == "slashes":
        print(get_with_slashes(date))
    else:
        raise ValueError("Unknown format")

today = datetime.date.today()
nice = get_date_nice(today)
print(nice)
print(formats)

November 12, 2024
['nice', 'slashes']


In [ ]:
def pprint(lis):
    for i in range(len(lis)):
        print(f"{i+1}: {lis[i]}")

if __name__ == "__main__":
    print("testing code:")
    pprint(['Cookies and Cream', 'Peanut Butter Cookie', 'Moose Tracks'])


In [ ]:
import csv
with open('my_csv.csv', 'r') as f:
    rows = list(csv.reader(f))

# Add a total column
for row in rows[1:]:
    row.append(str(float(row[1]) * int(row[2])))

# Writing the modified rows
with open('new_csv.csv', 'w') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)
